<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# THIS IS A CHISEL DEMONSTRATION FOR TIMA SLS TEAM.
# TO EXPERIMENT MORE ABOUT CHISEL, TAKE A LOOK AT [Introduction to Scala](1_intro_to_scala.ipynb)

# Chisel Demo
**Next: [Introduction à Scala](1_intro_to_scala.ipynb)**

Welcome! Perhaps you're an interested student who heard the name "Chisel" tossed about, or maybe you're a seasoned hardware design veteran who has been tasked by your manager to explore Chisel as a new HDL alternative. Either way if you are new to Chisel, you want to figure out as fast as possible what all the fuss is about. Look no futher - let's see what Chisel has to offer!

## Setup
Before we start, we need to download and imports the dependencies needed for the demo. 

**Please run the following two cell blocks by either pressing SHIFT+ENTER on your keyboard or the Run button in the menu**.

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.experimental._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

## Hardware Generators: Type-Safe Meta-Programming for RTL

All hardware description languages support writing single instances of an RTL design - Chisel is no different.
In fact, most Verilog/VHDL digital logic designs can be directly transcribed into Chisel!
While Chisel provides other awesome features that we will get to, we want to emphasize that users switching to Chisel will retain the exact same degree of control over their design as any other hardware language.

Take the following example of a 3-point moving average implemented in the style of a FIR filter.

<img src="https://raw.githubusercontent.com/freechipsproject/chisel3/master/doc/images/fir_filter.svg?sanitize=true" width="512" />

Chisel provides similar base primitives as synthesizable Verilog and *can* be used as such! Run next cell to declare our Chisel module.

In [ ]:
// 3-point moving average implemented in the style of a FIR filter
class MovingAverage3(bitWidth: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })

  val z1 = RegNext(io.in) // Create a register whose input is connected to the argument io.in
  val z2 = RegNext(z1)    // Create a register whose input is connected to the argument z1

  io.out := (io.in * 1.U) + (z1 * 1.U) + (z2 * 1.U) // `1.U` is an unsigned literal with value 1
}

After defining `class MovingAverage3`, let's instantiate it and take a look at its structure:

In [ ]:
// same 3-point moving average filter as before
visualize(() => new MovingAverage3(8))

In this visualization of the Chisel instance, the inputs on the left, and the z1 and z2 registers in gold. Both registers and io_in are multiplied by their coefficients and which are then added successively. The `tail` and `bits` elements are used to keep the additions from growing.

You may now ask: "Oh well and good - you can do stuff in Verilog in Chisel, but then why would I want to use Chisel?"

We are so glad you asked! The real power of Chisel comes from the ability to create **generators, not instances**. Suppose instead of only a `MovingAverage3` module, we wanted to create a generic `FIRFilter` module that is parameterized by a list of coefficients.

Below we have rewritten `MovingAverage3` to accept into a sequence of coefficients. The number of coefficients will determine the size of the filter.

In [ ]:
// Generalized FIR filter parameterized by the convolution coefficients
class FirFilter(bitWidth: Int, coeffs: Seq[Int], bin: Boolean=true) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })
  // Create the serial-in, parallel-out shift register
  val zs = Reg(Vec(coeffs.length, UInt(bitWidth.W)))
  zs(0) := io.in
  for (i <- 1 until coeffs.length) {
    zs(i) := zs(i-1)
  }

  // Do the multiplies
  val products = VecInit.tabulate(coeffs.length)(i => zs(i) * coeffs(i).U)

  // Sum up the products
  if (bin) {  io.out := products.reduceTree(_ +& _) }
  else { io.out := products.reduce(_ +& _) }
}

Now by changing our `coeffs` parameters during instantiation, our `FIRFilter` module can be used to instantiate an endless number of different hardware modules! Below we create the exact same moving average, but in a generic way !

In [ ]:
// same 3-point moving average filter as before
visualize(() => new FirFilter(8, Seq.fill(3)(1)))

## Built-in constructs
We can also create two different instances of `FIRFilter`, using a binary reduce tree, or a linear one, with the constructs embedded in Chisel. Here, we show the difference between `reduce` and `reduceTree`, as well as the ease one can evolve its design to match constraints.

In [ ]:
// same 8 coeffs filter, with and with binary reduce tree
visualize(() => new FirFilter(8, Seq.tabulate(8)(x => x%4), true))
visualize(() => new FirFilter(8, Seq.tabulate(8)(x => x%4), false))

## Type Genericity using Chisel
You can also change the **type** of samples in your design, to make a generic FIR filter on any type (here implementing the *Num[T]* trait, i.e. the +, \*, -, ... operations), with any number of coefficients.

In [ ]:
// Generalized FIR filter parameterized by the convolution coefficients AND the type of elements !
class GenFirFilter[T <: Data with Num[T]](gen: T, coeffs: Seq[Int]) extends Module {
  val io = IO(new Bundle {
    val in = Input(gen.cloneType)
    val out = Output(gen.cloneType)
  })
  // Create the serial-in, parallel-out shift register
  val zs = Reg(Vec(coeffs.length, gen.cloneType))
  zs(0) := io.in
  for (i <- 1 until coeffs.length) {
    zs(i) := zs(i-1)
  }

  // Do the multiplies
  val products = VecInit.tabulate(coeffs.length)(i => zs(i) * coeffs(i).S.asTypeOf(gen))

  // Sum up the products
  io.out := products.reduceTree(_ + _)
}

This way, you can define multiple hardware from the same description

In [ ]:
// Generic FIR filter, on 8 bit unsigned numbers, 12 bits signed numbers and 16 bits fixed point numbers
visualize(() => new GenFirFilter(UInt(8.W), Seq.tabulate(4)(x => x%4)))
visualize(() => new GenFirFilter(SInt(12.W), Seq.tabulate(4)(x => x%4)))
visualize(() => new GenFirFilter(FixedPoint(16.W, 8.BP), Seq.tabulate(4)(x => x%4)))

Without this powerful parameterization, we would need many more module definitions, likely one for each of these FIR filters. Ideally, we want our generators to be (1) composable, (2) powerful, and (3) enable fine-grained control over the generated design.

The benefits of Chisel are in how you use it, not in the language itself.
If you decide to write instances instead of generators, you will see fewer advantages of Chisel over Verilog.
But if you take the time to learn how to write generators, then the power of Chisel will become apparent and you will realize you can never go back to writing Verilog.
Learning to write generators is difficult, but we hope this tutorial will pave the way for you to become a better hardware designer, programmer, and thinker!

## Testing using Chisel
Indeed, easing hardware development is not enough to be broadly accepted as a replacement for usual RTL, such as VHDL, Verilog or SystemVerilog.
We need to be able to test our designs, still benefying from Chisel genericity, and from Scala features to build design verification.

First of all, we need to design a software implementation of the algorithm, for comparison and validation purpose.

In [ ]:
// Software FIR filter
class SoftFirFilter(samples: Seq[Int], coeffs: Seq[Int]) {
    private val buffer = Array.fill(coeffs.length)(0)
    private var index  = 0
    
    private def slideWindow: Unit= {
        for (i <- buffer.length - 1 to 1 by -1){
            buffer(i) = buffer(i-1)
        }
        buffer(0) = if (index < samples.length) { samples(index) } else { 0 }
        index = index + 1
    }
    
    private def apply: Int = {
        slideWindow
        (buffer zip coeffs).map{case (s, c) => s*c}.reduce(_+_)
    }
    
    def compute: Seq[Int] = {
        samples.map(_ => apply) ++ coeffs.map(_ => apply)
    }
}

Then, we can design multiple testers for our design, with different sets of parameters

In [ ]:
import org.scalatest._
import chiseltest._

class IntFIRSpec extends FlatSpec with ChiselScalatestTester with Matchers {
    behavior of "Testers2"
    
    val debug = true
    
    private def runUInt(c: GenFirFilter[UInt], samples: Seq[Int], coeffs: Seq[Int]) = {
        val ref = new SoftFirFilter(samples, coeffs).compute
        if (debug) { println(ref) }
        for(i <- 0 until samples.length) {
            c.io.in.poke(samples(i).U)
            c.clock.step(1)
            c.io.out.expect(ref(i).U)
            if (debug) { println(s"Poked ${samples(i)}, expected ${ref(i)}, got ${c.io.out.peek}")}
           }
        // Empty filter
        for (i <- 0 until coeffs.length) {
            c.clock.step(1)
            c.io.out.expect(ref(samples.length+i).U)
        }
    }
    
    it should "respond correctly to a software dirac with UInt values" in {
        val triangle = Seq(1, 2, 3, 2, 1)
        val dirac = Seq(1, 0, 0, 0, 0)
        test(new GenFirFilter(UInt(32.W), triangle)){
            c => runUInt(c, dirac, triangle)
        }
    }
    
    it should "respond correctly to a software step with UInt values" in {
        val triangle = Seq(1, 2, 3, 2, 1)
        val step = Seq(0, 1, 1, 1, 1, 1, 1)
        test(new GenFirFilter(UInt(32.W), triangle)){
            c => runUInt(c, step, triangle)
        }
    }
    
    private def runSInt(c: GenFirFilter[SInt], samples: Seq[Int], coeffs: Seq[Int]) = {
        val ref = new SoftFirFilter(samples, coeffs).compute
        for(i <- 0 until samples.length) {
            c.io.in.poke(samples(i).S)
            c.clock.step(1)
            c.io.out.expect(ref(i).S)
            if (debug) { println(s"Poked ${samples(i)}, expected ${ref(i)}, got ${c.io.out.peek}")}
        }
    }
    
    it should "respond correctly to a random signed values" in {
        val r = scala.util.Random
        val coeffs = Seq.fill(10)(r.nextInt(10)-r.nextInt(10))
        val samples = Seq.fill(100)(r.nextInt(10)-r.nextInt(10))
        test(new GenFirFilter(SInt(32.W), coeffs)){
            c => runSInt(c, samples, coeffs)
        }
    }
    
    it should "fail, to show you how it is !" in {
        test(new GenFirFilter(UInt(32.W), Seq(1, 2))){
            c => c.io.out.expect(4.U)
        }
    }
}

In [ ]:
run(new IntFIRSpec)

---
# All done!

[Return to the top.](#top)